In [ ]:
from collections import OrderedDict
import numpy as np

from astropy.table import vstack
import astropy.units as u
from astropy.io import ascii
import astropy.coordinates as coord
import gala.coordinates as gc

In [ ]:
a13_mg = ascii.read("../data/a13_li17_mgiants.csv")
# HACK because...why
ra = coord.Angle(a13_mg['RA'], unit=u.hourangle)
dec = coord.Angle(a13_mg['DEC'], unit=u.degree)
gal = coord.SkyCoord(ra=ra, dec=dec).galactic
a13_mg['l'] = gal.l.degree
a13_mg['b'] = gal.b.degree
a13_mg.remove_column('RA')
a13_mg.remove_column('DEC')
a13_mg.rename_column('v_helio_final', 'v_hel')
a13_mg['v_err'] = [5.5]*len(a13_mg)
a13_mg['structure'] = ['A13']*len(a13_mg)
a13_mg['tracer'] = ['MG']*len(a13_mg)

a13_rr = ascii.read("../data/a13_rrlyrae.csv")
a13_rr.remove_column('vsys_gsr')
a13_rr.rename_column('vsys_hel', 'v_hel')
a13_rr.rename_column('evsys', 'v_err')
a13_rr.rename_column('struct', 'structure')
a13_rr['tracer'] = ['RRL']*len(a13_rr)

In [ ]:
gass_mg = ascii.read("../data/mongass_crane03_mgiants.csv")
ra = coord.Angle(gass_mg['RA'], unit=u.hourangle)
dec = coord.Angle(gass_mg['Dec'], unit=u.degree)
gal = coord.SkyCoord(ra=ra, dec=dec).galactic
gass_mg['l'] = gal.l.degree
gass_mg['b'] = gal.b.degree
gass_mg.remove_column('RA')
gass_mg.remove_column('Dec')
gass_mg.rename_column('d', 'dist')
gass_mg.remove_column('v_gsr')
gass_mg['structure'] = ['Mon/GASS']*len(gass_mg)
gass_mg['tracer'] = ['MG']*len(gass_mg)

gass_rr = ascii.read("../data/mongass_rrlyrae.csv")
gass_rr.remove_column('vsys_gsr')
gass_rr.rename_column('vsys_hel', 'v_hel')
gass_rr.rename_column('evsys', 'v_err')
gass_rr.rename_column('struct', 'structure')
gass_rr['structure'] = ['Mon/GASS']*len(gass_rr)
gass_rr['tracer'] = ['RRL']*len(gass_rr)

In [ ]:
triand_mg = ascii.read("../data/triand_sheffield14_mgiants.csv")
triand_mg = triand_mg[triand_mg['dwarf_giant'] == 'G']
triand_mg.rename_column('vhel', 'v_hel')
triand_mg.rename_column('verr', 'v_err')
triand_mg.remove_column('ID')
triand_mg.remove_column('S/N')
triand_mg.remove_column('EW1')
triand_mg.remove_column('EW2')
triand_mg.remove_column('UT')
triand_mg.remove_column('INST')
triand_mg.remove_column('class')
triand_mg['structure'] = ['TriAnd']*len(triand_mg)
triand_mg['tracer'] = ['MG']*len(triand_mg)

triand_rr = ascii.read("../data/triand_price-whelan15_rrlyrae.csv")
gal = coord.SkyCoord(ra=triand_rr['ra']*u.degree, dec=triand_rr['dec']*u.degree).galactic
triand_rr['l'] = gal.l.degree
triand_rr['b'] = gal.b.degree
triand_rr.rename_column('name', 'target')
triand_rr.rename_column('Vsys', 'v_hel')
triand_rr.rename_column('Err', 'v_err')
triand_rr.remove_column('v_ally')
triand_rr.remove_column('verr_ally')
triand_rr.remove_column('ra')
triand_rr.remove_column('dec')
triand_rr['structure'] = ['TriAnd']*len(triand_rr)
triand_rr['tracer'] = ['RRL']*len(triand_rr)

In [ ]:
all_data = vstack((a13_mg, a13_rr, gass_mg, gass_rr, triand_mg, triand_rr))

In [ ]:
len(all_data)

## Compute GSR velocities for all stars

In [ ]:
c = coord.Galactic(l=all_data['l']*u.degree, b=all_data['b']*u.degree)
v_gsr = gc.vhel_to_vgsr(c, all_data['v_hel']*u.km/u.s,
                        vcirc=236.*u.km/u.s,
                        vlsr=[11.1, 12.24, 7.25]*u.km/u.s)
all_data['v_gsr'] = v_gsr.value

In [ ]:
mg = all_data[all_data['tracer'] == 'MG']
rr = all_data[all_data['tracer'] == 'RRL']

In [ ]:
plt.scatter(mg['l'], mg['v_gsr'])
plt.scatter(rr['l'], rr['v_gsr'])

In [ ]:
plt.hist(rr['v_gsr'], bins='auto');

In [ ]:
all_data['l'].unit = u.degree
all_data['b'].unit = u.degree
all_data['v_hel'].unit = u.km/u.s
all_data['v_err'].unit = u.km/u.s
all_data['v_gsr'].unit = u.km/u.s

In [ ]:
all_data.write('../data/all_mg_rr.ecsv', format='ascii.ecsv', overwrite=True)